#### Name: Zhornichenko Ilya Alekseevich
#### Group: TMSS
#### Higher school of economics
#### Implementation of Entropy-complexity plane for featuring raw data

In [8]:
"""importing libraries"""
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import sys
import warnings
import random
from itertools import permutations
from scipy.special import xlogy
import math
import tqdm


In [9]:
def chaos_generator(time, setup_iter):
    time_init      = 0
    time_terminate = time
    number_of_iters = setup_iter
    """measuring delta t"""
    dt =  (time_terminate - time_init)/ number_of_iters
    """for-loop for time series"""
    time = []
    set_random_points = []
    for i in np.arange(time_init , time_terminate - dt, dt): 
        time.append(i)
        set_random_points.append(random.random()) 


    return time, set_random_points

In [10]:

def bandt_pompe(x, D):
    orders = list(permutations(list(range(D))))
    ord2idx = {o: i for i, o in enumerate(orders)}
    
    fac = math.factorial(D)
    dist = np.zeros(fac)
    denom = 0
    
    for s in tqdm(range(len(x) - D + 1)):
        cur = x[s:s + D]
        perm = tuple(np.argsort(cur))
        dist[ord2idx[perm]] += 1
        denom += 1
    return dist / denom

def entropy(dist):
    return -xlogy(dist, dist).sum()

In [14]:
def get_entropy_complexity(x, D):
    p_ts = bandt_pompe(x, D)
    fac = math.factorial(D)
    p_e = np.ones(fac) / fac
    p0 = np.zeros(fac); p0[0] += 1
    S_p = entropy(p_ts); S_pe = entropy(p_e)
    H_S = S_p / S_pe
    
    
    
    Q_zero = entropy((p0 + p_e) / 2) - entropy(p0) / 2 - S_pe / 2
    Q_j = (entropy((p_ts + p_e) / 2) - S_p / 2 - S_pe / 2) / Q_zero
    return H_S, Q_j * H_S

In [7]:

ent_chaos, comp_chaos = get_entropy_complexity(np.array([chaos_generator(1000000,200)[1]]), D=5)
#ent_lorentz, comp_lorentz = get_entropy_complexity(lorentz.values, D)
plt.figure(figsize=(5, 5))
plt.xlim(0, 1)
plt.ylim(0, 1)

plt.scatter(ent_chaos, comp_chaos, label='Chaos')
#plt.scatter(ent_lorentz, comp_lorentz, label='Lorentz')
plt.xlabel('Entropy HS')
plt.ylabel('Complexity CJS')
plt.title('Entropy-Complexity plane')
plt.legend(); plt.show()